# 导入函数库

In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras  # 在TF2.0版本中,已经是自带Keras了,所以不需要额外安装
import tensorflow_model_optimization as tfmot  # 导入TF2.0的模型优化函数库,降低模型优化难度,相当于调用借口解决问题
import zipfile
import tempfile
import tensorflow_datasets as tfds # 这个是之前说过的Tensorflow Datasets
%load_ext tensorboard

# 如果出现显存不够的错误,把这个代码加上

可以展示下不加这个出现错误的情形

In [2]:
# 加入下面这个代码片段主要是因为TF2.0对RTX20系列显卡支持不是很好,容易爆显存,所以设置成用多少占多少显存,而不是一次性全占了(默认)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 定义网络结构

## 一些参数设置

In [3]:
layers = tf.keras.layers
models = tf.keras.models
keras_utils = tf.keras.utils

In [4]:
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/'
                       'releases/download/v0.2/'
                       'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')  # 此处的预训练模型不带全连接层

## ResNet50模型改写用于Cifar10分类

In [5]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size,
                      padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x

In [6]:
def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(2, 2)):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1), strides=strides,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = layers.Conv2D(filters3, (1, 1), strides=strides,
                             kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

In [7]:
# 关于这里把官方ResNet50函数拿了出来,原因是这样的,我们知道Keras主要有Sequential model和Functional model两种构建模型方式,
# 而在进行迁移学习的时候，我们经常是加载预训练的模型(设置include_top=False),此时我们很常见的做法是add layer到模型上,
# 这样会形成模型嵌套,这个在使用global_pruning的时候会出错,所以我把ResNet50函数拿了出来,手动改了下
def ResNet50(include_top=False,
         weights='imagenet',
         input_shape=(250, 250, 3),
         pooling='avg',
         classes=3,
         **kwargs):
    img_input = layers.Input(shape=input_shape)  # 输入节点
    
    bn_axis = 3
    x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    
    if pooling == 'avg':
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(128, activation='relu', name='fc1')(x)
        x = layers.Dense(classes, activation='softmax', name='predictions')(x) # 此处是10分类，而不是ImageNet的1000分类
    
    inputs = img_input  # inputs是输入节点, x是输出节点
    model = models.Model(inputs, x , name='vgg16')  # 生成一个Model, 需要指定输入和输出
    
    weights_path = keras_utils.get_file(
        'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',  # 此处加载的是不存在全连接层的预训练模型
        WEIGHTS_PATH_NO_TOP,
        cache_subdir='models',
        file_hash='6d6bbae143d832006294945121d1f1fc')
    model.load_weights(weights_path, by_name=True)  
    # 加载在ImageNet上预训练过的模型，注意by_name参数很有用，把layer和layer name对应上了
    
    return model

## 声明一个ResNet50模型实例

In [8]:
model = ResNet50()
# model.load_weights('./baseline.h5')

## 这个函数可以用于查看网络结构和参数量

In [9]:
model.summary()

Model: "vgg16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 125, 125, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 125, 125, 64) 256         conv1[0][0]                      
______________________________________________________________________________________________

In [10]:
log_dir = './logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

## 对特征提取层进行冻结,加快模型训练速度

可以展示下不加的情形，更直观的说明为什么需要训练更长时间

In [11]:
length = len(model.layers)  # 查看模型有多少层Layer
print(length)

178


In [12]:
# 对模型的特征提取层进行冻结,可以加快模型训练速度
# for i in range(length - 2): # 我们看下上面的model.summary()，可以发现最后2层是全连接，所以不冻结
#     model.layers[i].trainable = False # 对该层Layer进行冻结

In [13]:
for i in range(length):
    print(model.layers[i].trainable)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


## 进行模型训练

In [14]:
def convert(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
    image = tf.image.resize_with_crop_or_pad(image, 256, 256) # Add 6 pixels of padding
    image = tf.image.random_crop(image, size=[250, 250, 3]) # Random crop back to 32x32
    return image, label

def augment(image,label):
    image,label = convert(image, label)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    return image,label

In [15]:
# 此处使用tfds的方式构建data pipeline
(raw_test, raw_train, raw_validation), metadata = tfds.load(
    'beans', # 数据集名称，这个是手势识别分类数据集，共3个类别
    split=['test', 'train', 'validation'], # 这里的raw_test和split的'test'对应，raw_train和split的'train'对应
    with_info=True, # 这个参数和metadata对应
    as_supervised=True, # 这个参数的作用是返回tuple形式的(input, label),举个例子，raw_test=tuple(input, label)
    shuffle_files=True  # 对数据进行扰乱操作，可以自己体会下设置成False时，下面imshow的时候的结果差别
)

IMG_SIZE = 250 # All images will be resized to 300X300

BATCH_SIZE = 16
SHUFFLE_BUFFER_SIZE = 1034

# 可以体验下这里是否加prefetch(tf.data.experimental.AUTOTUNE)和cache()的区别，对训练速度，以及CPU负载有影响
train_batches = raw_train.shuffle(SHUFFLE_BUFFER_SIZE, reshuffle_each_iteration=True).map(augment).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_batches = raw_test.map(convert).batch(BATCH_SIZE)

# compile模型并训练
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [16]:
model.fit(
    train_batches,
    epochs=20,
    callbacks=[tensorboard_callback]
)

Epoch 1/20
65/65 [==============================] - 11s 165ms/step - loss: 1.0938 - accuracy: 0.3956
Epoch 2/20
65/65 [==============================] - 9s 139ms/step - loss: 1.0572 - accuracy: 0.4565
Epoch 3/20
65/65 [==============================] - 9s 139ms/step - loss: 1.0129 - accuracy: 0.5329
Epoch 4/20
65/65 [==============================] - 9s 139ms/step - loss: 0.9777 - accuracy: 0.5493
Epoch 5/20
65/65 [==============================] - 9s 140ms/step - loss: 0.9593 - accuracy: 0.5735
Epoch 6/20
65/65 [==============================] - 9s 141ms/step - loss: 0.9451 - accuracy: 0.5870
Epoch 7/20
65/65 [==============================] - 9s 144ms/step - loss: 0.9303 - accuracy: 0.5967
Epoch 8/20
65/65 [==============================] - 9s 141ms/step - loss: 0.9240 - accuracy: 0.6122
Epoch 9/20
65/65 [==============================] - 9s 141ms/step - loss: 0.9107 - accuracy: 0.6267
Epoch 10/20
65/65 [==============================] - 9s 140ms/step - loss: 0.9104 - accuracy: 0.619

## Baseline的test acc和推断速度

In [17]:
# 此处我们可以看到Baseline的test acc和inference速度为6ms/step
_, baseline_model_accuracy = model.evaluate(test_batches, verbose=1)
print('Baseline test accuracy: ', baseline_model_accuracy)

keras_file = './test.h5'
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to: ', keras_file)

8/8 [==============================] - 0s 35ms/step - loss: 0.8701 - accuracy: 0.6797
Baseline test accuracy:  0.6796875
Saved baseline model to:  ./test.h5


In [18]:
%tensorboard --logdir ./logs

Reusing TensorBoard on port 6010 (pid 20157), started 3:17:58 ago. (Use '!kill 20157' to kill it.)